In [1]:

#Procedimiento realizado a la tabla precios_semana_20200518.csv
#   1.Se borran los datos repetidos si existen y los valores null

import pandas as pd

df_ps_20200503 = pd.read_json("Datasets\precios_semana_20200503.json")
df_ps_20200503.info()
df_ps_20200503.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397734 entries, 0 to 397733
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   precio       397734 non-null  object
 1   producto_id  397734 non-null  object
 2   sucursal_id  397734 non-null  object
dtypes: object(3)
memory usage: 9.1+ MB


,precio,producto_id,sucursal_id
28831,55.9,7441008157472,49-1-2
54678,141,7622210649287,9-3-5251
321568,58,7795513043271,13-1-52
140754,97.99,7790520016430,7-1-35
362217,285.9,7798143960345,45-1-5


In [2]:
#-------------Tratamiento General
# Se borran valores repetidos
df_ps_20200503.drop_duplicates(inplace=True)

#Se borran valores nulos
df_ps_20200503.dropna(inplace=True)

#-----------Manejo de los Id del columna sucursal_id
#Se convierte el codigo string en una lista con 
# los subcodigos (comercioId-banderaId-sucursalId.) 
Ids =[]
for id_Tex in df_ps_20200503["sucursal_id"]:
    Ids.append(id_Tex.split("-")) 
    
#Se procede a crear la nueva columna con los nuevos Id_Sucursal
Id_Sucursal =[]
for Idcompuesta in Ids:
    num = int(Idcompuesta[0])*10000*100  +  int(Idcompuesta[1])*10000 + int(Idcompuesta[2])
    Id_Sucursal.append(num)
    
#Se inserta la nueva columna y se borra la anterior
df_ps_20200503.insert(0,"Id_Sucursal", Id_Sucursal, allow_duplicates=False)
df_ps_20200503.drop("sucursal_id",axis=1,inplace=True)


#------------Manejo de los Id del columna producto_id
#Se verifica el producto_id creando lista para la columna con valores Id 
# que se puedan convertir en int 
id_producto =[]
for i in range(len(df_ps_20200503)):
    try:
        id_producto.append(int(df_ps_20200503.iloc[i,1]))
    except ValueError:
        id_producto.append(None)
        
#Se agrega la columna ID_producto con valores de id int
df_ps_20200503.insert(loc = 0 , column = "Id_producto", value = id_producto)

#Se guardan valores con problemas, con mas tiempo se pueden mejorar e integrar a la tabla
df_ps_20200503_erroresId = df_ps_20200503[df_ps_20200503["Id_producto"].isnull()].drop(["Id_producto"], axis=1)

#Se borran valores nulos o con problemas
df_ps_20200503.dropna(subset="Id_producto",inplace=True)

#Se elimina la columna producto_id
df_ps_20200503.drop(["producto_id"], axis=1,inplace=True)


#------------Manejo de la columna precios



precio =[]
for i in range(len(df_ps_20200503)):
    try:
        precio.append(int(df_ps_20200503.iloc[i,0]))
    except ValueError:
        precio.append(None)
        
#Se agrega la columna precio_nuevo con valores de id int
df_ps_20200503.insert(loc = 2 , column = "precio_nuevo", value = precio )

#Se guardan valores con problemas, con mas tiempo se pueden mejorar e integrar a la tabla
df_ps_20200503_erroresPrecio = df_ps_20200503[df_ps_20200503["precio_nuevo"].isnull()].drop(["precio_nuevo"], axis=1)

#Se borran valores nulos o con problemas
df_ps_20200503.dropna(subset="precio_nuevo",inplace=True)

#Se elimina la columna precio vieja y se remplaza por la nueva
df_ps_20200503.drop(["precio"], axis=1,inplace=True)
df_ps_20200503.rename(columns={'precio_nuevo': 'precio'})


#Se exporta la base de datos a MySQL
import sqlalchemy as db
database_username='root'            # Nombre de cliente en MySQL Workbrench
database_password='JCSR.MySQL123'   # Contraseña de MySQL Workbrench
database_ip='localhost'
database_name='ETL_Lab01'           # Nombre de Base de datos a donde nos conectaremos (se debio crear primero en Mysql)
database_conection = db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}',
                                    encoding='latin1')
coneccion=database_conection.connect()
df_ps_20200503.to_sql('ps_20200503',coneccion, index=False)

#Creacion de conexion con Mysql 
import pymysql
conexion = pymysql.connect(
   host = 'localhost',
   user = 'root',
   passwd='JCSR.MySQL123',
   db='ETL_Lab01'           #LA base de datos se debio crear primero en Mysql
)
cursor = conexion.cursor()
try:
    #Se crean las relaciones
    cursor.execute("SET FOREIGN_KEY_CHECKS=0;")
    cursor.execute("alter table ps_20200503 add foreign key (Id_Sucursal) references sucursal(Id_Sucursal);")
    cursor.execute("alter table ps_20200503 add foreign key (Id_producto) references producto(Id_producto);")
    cursor.execute("SET FOREIGN_KEY_CHECKS=1;")
finally:
    conexion.close()
print("Carga de tabla 'ps_20200503' a MySQL completa")

Carga de tabla 'ps_20200503' a MySQL completa
